In [56]:
import csv
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
pathAnn = "/Users/yayafung/Desktop/opp/OPP-115/annotations"
os.chdir(pathAnn)
aDataset = []  # {index: [label, text]}
x = []
y = []
typeIndex = 5
itemIndex = 6
start = len('"selectedText": "')
end = -2


In [57]:
def tfIdf(dataset):
    tfIdfVectorizer = TfidfVectorizer(
        stop_words='english', use_idf=True, ngram_range=(1, 3))
    tfIdf = tfIdfVectorizer.fit_transform(dataset.values())
    df = pd.DataFrame(tfIdf[0].T.todense(
    ), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    return df


def readCsv(file_path):
    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        label = 0
        for row in csv_reader:
            if (row[4] == "0"):
                label = 1
            else:
                label = 0
            text = re.findall(
                f'"selectedText"\:(.*?)"\,', row[6])
            if len(text) > 0:
             #   print((row[4] == "0"), label, text)
                for t in text:
                    aDataset.append([label, t])
                    y.append(label)
                    x.append(t)


In [58]:
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".csv"):
        file_path = f"{pathAnn}/{file}"
        readCsv(file_path)


In [59]:
df = pd.DataFrame(aDataset, columns=['label', 'text'])
# print(df)
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=0)

text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(
    1, 2), lowercase=True)
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)
# print(y_test)


In [60]:
logit = LogisticRegression(
    C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4, max_iter=400)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
cv_results = cross_val_score(
    logit, X_train_text, y_train, cv=skf, scoring='f1_micro')
print(cv_results, cv_results.mean())
logit.fit(X_train_text, y_train)
test_preds = logit.predict(X_test_text)


[0.99599654 0.99615884 0.99680805 0.99545528 0.99577991] 0.9960397233844308


In [61]:
count = 0
for i in test_preds:
    if i == 1:
        count += 1
print(count)
print(len(test_preds))


46
10269


In [62]:
count = 0
for i in y_test:
    if i == 1:
        count += 1
print(count)
print(len(test_preds))


69
10269


In [63]:
count = 0
for i in range(len(y_test)):
    if y_test[i] == test_preds[i]:
        count += 1
print(len(y_test)-count)


25


In [64]:
X_train_text


<92418x60426 sparse matrix of type '<class 'numpy.float64'>'
	with 780978 stored elements in Compressed Sparse Row format>

In [103]:
pathPolicy = "/Users/yayafung/Desktop/opp/scraped_policies"
os.chdir(pathPolicy)
lines = []
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        print(file)
        file_path = f"{pathPolicy}/{file}"
        with open(file_path) as f:
            lines = lines + f.readlines()


adarshurs.android.vlcmobileremote.txt
a2z.Mobile.Event6243.txt


In [104]:
len(lines)

287

In [105]:
v_lines = text_transformer.transform(lines)
preds = logit.predict(v_lines)

In [ ]:
for u in range(len(y_test)):
    if y_test[u] == 1:
        print(u)

In [106]:
len(preds)

287

In [107]:
preds

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [108]:
def read_txt(file):
    with open(file_path) as f:
        text = f.read()
        lines = f.readlines()
    return text, lines

def remove_other(file):
    text, lines = read_txt(file)
    
    return text


remove_other(
    "/Users/yayafung/Desktop/opp/scraped_policies/adarshurs.android.vlcmobileremote.txt")

    



ValueError: Found array with 0 sample(s) (shape=(0, 60426)) while a minimum of 1 is required by TfidfTransformer.